In [ ]:
# server_probs_agg.py
import flwr as fl
from typing import List, Dict, Tuple

def agg_eval_metrics(metrics: List[Dict]) -> Dict:
    # metrics 形如 [{"probs_sum":[...], "count":N, "acc":...}, ...]
    total = sum(m["count"] for m in metrics if "count" in m)
    if total == 0:
        return {}
    C = len(metrics[0]["probs_sum"])
    summed = [0.0]*C
    for m in metrics:
        ps = m.get("probs_sum")
        if ps is None: 
            continue
        for i in range(C):
            summed[i] += ps[i]
    avg_probs = [x/total for x in summed]  # 全局平均概率（可存盘用于后续融合）
    # 也可把 acc 做样本加权平均：
    acc = sum(m.get("acc",0.0)*m.get("count",0) for m in metrics)/total
    return {"avg_probs": avg_probs, "global_acc_weighted": acc, "N_total": total}

strategy = fl.server.strategy.FedAvg(
    # 跳过“参数意义”，只借壳通信与metrics聚合
    min_available_clients=2,
    min_fit_clients=2,
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    evaluate_metrics_aggregation_fn=agg_eval_metrics,
)

fl.server.start_server(
    server_address="0.0.0.0:8080",
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=3),
)


	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: num_rounds=3, no round_timeout
INFO :      Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
